In [1]:
from langchain.globals import set_debug
set_debug(True)

## Basics

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-pro")
llm.invoke("지구의 자전 주기는?")

[llm/start] [llm:ChatGoogleGenerativeAI] Entering LLM run with input:
{
  "prompts": [
    "Human: 지구의 자전 주기는?"
  ]
}
[llm/end] [llm:ChatGoogleGenerativeAI] [1.82s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "약 24시간",
        "generation_info": {
          "finish_reason": "STOP",
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE",
              "blocked": false
            },
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE",
              "blocked": false
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "NEGLIGIBLE",
              "blocked": false
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "NEGLIGIBLE",
              "blocked": false
            }
         

AIMessage(content='약 24시간', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-37bf4f45-011a-436a-8f95-8d7b08c9cb80-0')

### PromptTemplate 적용

In [3]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("You are an expert in astronomy. Answer the question. <Question>: {input}")
prompt

ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='You are an expert in astronomy. Answer the question. <Question>: {input}'))])

In [4]:
llm = ChatGoogleGenerativeAI(model="gemini-pro")

# Chain 연결 (LangChain Expression Language)
chain = prompt | llm

chain.invoke({"input": "지구의 자전 주기는?"})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "지구의 자전 주기는?"
}
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "input": "지구의 자전 주기는?"
}
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatGoogleGenerativeAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an expert in astronomy. Answer the question. <Question>: 지구의 자전 주기는?"
  ]
}
[llm/end] [chain:RunnableSequence > llm:ChatGoogleGenerativeAI] [2.91s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "24시간",
        "generation_info": {
          "finish_reason": "STOP",
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE",
              "blocked": false
            },
            {
              "category

AIMessage(content='24시간', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-1bec520c-f264-4ab8-b4e4-21fb8c9e2202-0')

## Multi-Chain

In [8]:
from langchain_core.output_parsers.string import StrOutputParser

prompt1 = ChatPromptTemplate.from_template("translates {korean_word} to English.")
prompt2 = ChatPromptTemplate.from_template("explain {english_word} using oxford dictionary to me in Korean.")

llm = ChatGoogleGenerativeAI(model="gemini-pro")

chain1 = prompt1 | llm | StrOutputParser()

chain2 = (
    {"english_word": chain1}
    | prompt2
    | llm
    | StrOutputParser()
)

chain2.invoke({"korean_word": "미래"})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "korean_word": "미래"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<english_word>] Entering Chain run with input:
{
  "korean_word": "미래"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<english_word> > chain:RunnableSequence] Entering Chain run with input:
{
  "korean_word": "미래"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<english_word> > chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "korean_word": "미래"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<english_word> > chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > chain:RunnableParallel<english_word> > chain:RunnableSequence > llm:ChatGoogleGenerativeAI] Entering LLM run with input:
{
  "prompts": [
    "Human: translates 미래 to English."
  ]
}
[llm/end] [chain:Runnable

'옥스포드 사전에서 설명한 미래\n\n미래는 현재 이후에 올 때를 말합니다. 미래는 우리가 영향을 미칠 수 있는 잠재적인 사건과 가능성이 있는 영역입니다. 미래를 예측하는 것은 불가능하지만, 현재의 행동과 결정이 미래에 어떤 영향을 미칠지 고려하는 것은 중요합니다.\n\n미래는 희망과 가능성, 불확실성과 불안의 시간이 될 수 있습니다. 우리가 선택하는 길은 미래의 모습을 형성하는 데 중요한 역할을 합니다.'